# Character based RNN language model trained on 'The Complete Works of William Shakespeare'
(c) Deniz Yuret, 2018. Based on http://karpathy.github.io/2015/05/21/rnn-effectiveness. **TODO:** Add julia/base example.

* Objectives: Learn to define and train a character based language model and generate text from it. Minibatch blocks of text. Keep a persistent RNN state between updates.
* Prerequisites: [RNN basics](06.rnn.ipynb), minibatch, param, param0, RNN, dropout, train!, Adam, nll

In [1]:
using Pkg
for p in ("Knet","ProgressMeter")
    haskey(Pkg.installed(),p) || Pkg.add(p)
end

true

In [2]:
RNNTYPE = :lstm
BATCHSIZE = 256
SEQLENGTH = 100
INPUTSIZE = 168
VOCABSIZE = 84
HIDDENSIZE = 334
NUMLAYERS = 1
DROPOUT = 0.0
LR=0.001
BETA_1=0.9
BETA_2=0.999
EPS=1e-08
EPOCHS = 30
ENV["COLUMNS"]=92;

## Load and minibatch data

In [3]:
# Load 'The Complete Works of William Shakespeare'
using Knet: Knet
include(Knet.dir("data","gutenberg.jl"))
trn,tst,chars = shakespeare()
map(summary,(trn,tst,chars))

┌ Info: Downloading Gutenberg shaks12
└ @ Main /data/scratch/deniz/.julia/dev/Knet/data/gutenberg.jl:18
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5451k  100 5451k    0     0  2831k      0  0:00:01  0:00:01 --:--:-- 2830k


("4934845-element Array{UInt8,1}", "526731-element Array{UInt8,1}", "84-element Array{Char,1}")

In [4]:
# Print a sample
println(string(chars[trn[1020:1210]]...)) 


    Cheated of feature by dissembling nature,
    Deform'd, unfinish'd, sent before my time
    Into this breathing world scarce half made up,
    And that so lamely and unfashionable
 


In [5]:
# Minibatch data
using Knet: minibatch
function mb(a)
    N = length(a) ÷ BATCHSIZE
    x = reshape(a[1:N*BATCHSIZE],N,BATCHSIZE)' # reshape full data to (B,N) with contiguous rows
    minibatch(x[:,1:N-1], x[:,2:N], SEQLENGTH) # split into (B,T) blocks 
end
dtrn,dtst = mb.((trn,tst))
length.((dtrn,dtst))

(192, 20)

In [6]:
summary.(first(dtrn))  # each x and y have dimensions (BATCHSIZE,SEQLENGTH)

("256×100 Array{UInt8,2}", "256×100 Array{UInt8,2}")

## Define and initialize model

In [7]:
using Knet: param, param0, RNN, dropout

In [8]:
struct CharLM; input; rnn; output; end

CharLM(vocab::Int,input::Int,hidden::Int; o...) = 
    CharLM(Embed(vocab,input), RNN(input,hidden; o...), Linear(hidden,vocab))

function (c::CharLM)(x; pdrop=0, hidden=nothing)
    x = c.input(x)                # (B,T)->(X,B,T)
    x = dropout(x, pdrop)
    x = c.rnn(x, hidden=hidden)   # (H,B,T)
    x = dropout(x, pdrop)
    x = reshape(x, size(x,1), :)  # (H,B*T)
    return c.output(x)            # (V,B*T)
end

In [9]:
struct Embed; w; end

Embed(vocab::Int,embed::Int)=Embed(param(embed,vocab))

(e::Embed)(x) = e.w[:,x]

In [10]:
struct Linear; w; b; end

Linear(input::Int, output::Int)=Linear(param(output,input), param0(output))

(l::Linear)(x) = l.w * x .+ l.b

In [11]:
# For running experiments
using Knet: train!, Adam; import ProgressMeter
function trainresults(file,model)
    if (print("Train from scratch? ");readline()[1]=='y')
        updates = 0; prog = ProgressMeter.Progress(EPOCHS * length(dtrn))
        callback(J)=(ProgressMeter.update!(prog, updates); (updates += 1) <= prog.n)
        opt = Adam(lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)
        train!(model, dtrn; callback=callback, optimizer=opt, pdrop=DROPOUT, hidden=[])
        Knet.gc(); Knet.save(file,"model",model)
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model = Knet.load(file,"model")
    end
    return model
end

trainresults (generic function with 1 method)

In [12]:
clm = trainresults("shakespeare.jld2", 
        CharLM(VOCABSIZE, INPUTSIZE, HIDDENSIZE; rnnType=RNNTYPE, numLayers=NUMLAYERS, dropout=DROPOUT));

Train from scratch? stdin> y


Progress: 100%|█████████████████████████████████████████████████████| Time: 0:02:47m


In [13]:
using Knet: nll
exp(nll(clm,dtst))  # Perplexity

4.2027154f0

In [14]:
# Sample from trained model

function generate(model,n)
    function sample(y)
        p = Array(exp.(y)); r = rand()*sum(p)
        for j=1:length(p); (r -= p[j]) < 0 && return j; end
    end
    x = 1
    h = []
    for i=1:n
        y = model([x], hidden=h)
        x = sample(y)
        print(chars[x])
    end
    println()
end;

In [15]:
generate(clm,1000)

GakE. SENATOR on your Greenchias'.

Enter Othelas Francis! Queen to conque Brook Getengal
  
  PETRUCHIO. You have a church. Three rude fury on my purse,  
    So plead rymbly should be inviritstited,
    With some bodes the wonder's language mething.
  Jul. It is here like a holy credic from to know.
    And she will rend us there assare that!
    Had you know your meaning?
  CORIOLANUS. Virtue whee her tresamen divine, you mound her.
    How now, cousin, gods in one Hamlet,
    Offer me, and would not beherch?
  TROILUS. There in a censul. O, there needs he not.
    Be it after-'wickness thought, and dost to you.
    Well, willind may be than husband
    Cainterge thee and to plead me, maly; heard
    He proves much he is rage and banished.
    These this adders drops never not fit the field,
    And thus from their graces being in Rome.
    Enough thou sto'd and brief; within as teaching Marcius.
  GLOUCESTER. By my Romans; tell me: 'tis coldly some
    That en
